In [2]:
import dask.dataframe as dd

df = dd.read_parquet("../../data/georgia_data_2/georgia_complete_dataset.parquet")
df.head(10)


,time,icao24,callsign,latitude,longitude,altitude,velocity,heading,vertrate,onground
0,2025-01-24 03:34:54+00:00,ad67ae,<NA>,33.646747,-84.437313,<NA>,82.839862,268.9325,17.23136,False
1,2025-01-24 03:34:55+00:00,ad67ae,<NA>,33.646729,-84.438115,426.72,82.851042,268.576795,17.23136,False
2,2025-01-24 03:34:56+00:00,ad67ae,<NA>,33.646683,-84.439068,457.2,82.351221,268.210089,16.90624,False
3,2025-01-24 03:34:57+00:00,ad67ae,<NA>,33.646701,-84.439888,464.82,82.368895,267.852415,16.58112,False
4,2025-01-24 03:34:58+00:00,ad67ae,DAL3153,33.646637,-84.44103,487.68,82.389777,267.494907,16.256,False
5,2025-01-24 03:34:59+00:00,ad67ae,DAL3153,33.646607,-84.441833,510.54,81.875827,267.479172,15.93088,False
6,2025-01-24 03:35:00+00:00,ad67ae,DAL3153,33.646607,-84.442463,518.16,81.900066,267.119622,15.60576,False
7,2025-01-24 03:35:01+00:00,ad67ae,DAL3153,33.646545,-84.443777,525.78,81.900066,267.119622,15.28064,False
8,2025-01-24 03:35:02+00:00,ad67ae,DAL3153,33.646514,-84.444237,533.4,81.900066,267.119622,14.95552,False
9,2025-01-24 03:35:03+00:00,ad67ae,DAL3153,33.646468,-84.44521,571.5,81.958211,266.401232,14.6304,False


In [3]:
df.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 10 entries, time to onground
dtypes: bool[pyarrow](1), double[pyarrow](6), string(2), timestamp[ns, tz=UTC][pyarrow](1)

In [5]:
num_rows = df.shape[0].compute()
print(f"Number of rows: {num_rows}")


Number of rows: 48243364


In [5]:
# compute the number of unique values in icao24
unique_icao24 = df['icao24'].nunique().compute()
print(f"Number of unique icao24: {unique_icao24}")

# compute the number of unique values in callsign
unique_callsign = df['callsign'].nunique().compute()
print(f"Number of unique callsign: {unique_callsign}")


Number of unique icao24: 8614
Number of unique callsign: 11934


In [6]:
# how many unique icao24 and callsign combinations
df.groupby('icao24').size().compute()

icao24
04004c    3133
06804e    1569
06a123    7025
06a129    5133
0aa918     497
          ... 
cae20f     624
e4809f     444
e49d87    2260
e80212    6668
e80244    3723
Length: 8614, dtype: int64

In [4]:
# group by icao24 and callsign and agggregate latitude	longitude	altitude	velocity	heading	vertrate
flights_df = df.groupby(['icao24', 'callsign']).agg({'latitude': 'mean', 'longitude': 'mean', 'altitude': 'mean', 'velocity': 'mean', 'heading': 'mean', 'vertrate': 'mean'}).sample(frac=0.01).compute()

flights_df.head()

,,latitude,longitude,altitude,velocity,heading,vertrate
icao24,callsign,,,,,,
a87c12,FFT3955,32.471794,-81.478927,11857.120488,203.283372,196.221136,-0.011742
ab15c1,AAY1239,31.876884,-81.083961,11620.887215,177.640601,188.897428,-1.33289
a379d3,DAL798,33.641882,-84.43315,<NA>,<NA>,<NA>,<NA>
a086ce,EDV5532,33.651916,-84.481403,3061.25483,137.428518,283.178827,11.072975
a54c9f,TCN440,34.165028,-83.27917,11333.922806,171.496294,242.093612,3.042226


In [5]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 189 entries, ('a87c12', 'FFT3955') to ('a2b759', 'SWA1629')
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   latitude   189 non-null    double[pyarrow]
 1   longitude  189 non-null    double[pyarrow]
 2   altitude   188 non-null    double[pyarrow]
 3   velocity   188 non-null    double[pyarrow]
 4   heading    188 non-null    double[pyarrow]
 5   vertrate   188 non-null    double[pyarrow]
dtypes: double[pyarrow](6)
memory usage: 294.6+ KB


In [2]:

# Convert time column to datetime if not already
df['time'] = dd.to_datetime(df['time']).astype('datetime64[ns]')
df = df.set_index('time')

# Resample using Dask
resampled_df = df.resample('1T').agg({
    'icao24': 'first',
    'callsign': 'last',
    'latitude': 'last',
    'longitude': 'last',
    'altitude': 'mean',
    'velocity': 'mean',
    'heading': 'mean',
    'vertrate': 'mean',
    'onground': 'last'
}).reset_index()

# Compute the result and check the size reduction
original_size = df.shape[0].compute()
resampled_size = resampled_df.shape[0].compute()
compression_ratio = original_size / resampled_size

print(f"Original rows: {original_size:,}")
print(f"Resampled rows: {resampled_size:,}")
print(f"Compression ratio: {compression_ratio:.2f}x")

# Save the resampled data if desired
resampled_df.to_parquet("../../data/georgia_data_2/georgia_resampled_1min.parquet")

: 